In [1]:
import numpy as np
import torch
import torch.nn as nn

In [6]:
np.random.seed(100)

mean_set_QAM = np.asarray([-3 - 3j, -3 - 1j, -3 + 1j, -3 + 3j, -1 - 3j, -1 - 1j, -1 + 1j, -1 + 3j,
                           1 - 3j, 1 - 1j, 1 + 1j, 1 + 3j, 3 - 3j, 3 - 1j, 3 + 1j, 3 + 3j
                           ], dtype=np.complex64)

def generate_real_samples_with_labels_Rayleigh(number=100):
    h_r = np.random.normal(scale=np.sqrt(2) / 2, size=number)
    h_i = np.random.normal(scale=np.sqrt(2) / 2, size=number)
    h_complex = h_r + 1j * h_i
    labels_index = np.random.choice(len(mean_set_QAM), number)
    data = mean_set_QAM[labels_index]
    received_data = h_complex * data
    received_data = np.hstack(
        (np.real(received_data).reshape(len(data), 1), np.imag(received_data).reshape(len(data), 1)))
    gaussion_random = np.random.multivariate_normal([0, 0], [[0.01, 0], [0, 0.01]], number).astype(np.float32)
    received_data = received_data + gaussion_random
    conditioning = np.hstack((np.real(data).reshape(len(data), 1), np.imag(data).reshape(len(data), 1),
                              h_r.reshape(len(data), 1), h_i.reshape(len(data), 1))) / 3
    return received_data, conditioning



In [5]:
class generator_conditional(nn.Module):
    def __init__(self, z, conditioning):
        super(generator_conditional, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(z+conditioning,32),
            nn.ReLU(),
            nn.Linear(32,32),
            nn.ReLU(),
            nn.Linear(32,32),
            nn.ReLU(),
            nn.Linear(32,1)
        )

    def forward(self, input):
        self.main(input)

class discriminator_conditional(nn.Module):
    def __init__(self, X, conditioning):
        super(discriminator_conditional, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(X+conditioning, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)


In [8]:
data_size = 10000
data, one_hot_labels = generate_real_samples_with_labels_Rayleigh(data_size)


In [3]:
G = generator_conditional(2, 4)
print(G)

generator_conditional(
  (main): Sequential(
    (0): Linear(in_features=6, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [6]:
D = discriminator_conditional(16,4)
print(D)

discriminator_conditional(
  (main): Sequential(
    (0): Linear(in_features=20, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=2, bias=True)
    (7): Sigmoid()
  )
)


In [15]:
a = np.random.rand(32,6)

c = G(torch.tensor(a).to(torch.float32))
c

In [17]:
x = np.random.rayleigh(scale=2, size = (2,3))
x

array([[3.28057068, 0.99448694, 2.72099857],
       [4.17553159, 0.89816245, 2.73885692]])